<a href="https://colab.research.google.com/github/gabbyprecious/hng-conversationalAnalysisUsingGPT2/blob/master/BobbyX_GPT2_Convo_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import json
import random
import re
import pandas as pd
from numpy import int64

Using https://github.com/nshepperd/gpt-2 repo as a staring point for GPT finetuning (Tenoke repo here is a fork of nsheppered with extra CL option for adjustinglearning rate)

In [0]:
!git clone https://github.com/tenoke/gpt-2

Cloning into 'gpt-2'...
remote: Enumerating objects: 244, done.
remote: Total 244 (delta 0), reused 0 (delta 0), pack-reused 244
Receiving objects: 100% (244/244), 4.39 MiB | 16.34 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [0]:
os.chdir('gpt-2')

In [0]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 3.3MB/s 
     |████████████████████████████████| 604kB 10.1MB/s 
     |████████████████████████████████| 51kB 21.6MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=ebe5cb294d2b9d24ff152a09bfcc83b356893f45c19dd02a4bb7c9a9b319e7e5
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533179 sha256=4b1672b1ad84d49f9824773617352f6b8a374e5a3844a5a013d7908850820d0d
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 674kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 55.2Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 712kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:22, 62.5Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.96Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 45.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 50.0Mit/s]                                                       


In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bobbyml","key":"c8daf735623bcc549beb353ea12f692a"}'}

In [0]:
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus --force

 98% 782M/799M [00:05<00:00, 180MB/s]
100% 799M/799M [00:05<00:00, 157MB/s]


In [0]:
!unzip ubuntu-dialogue-corpus.zip

Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


In [0]:
!du -h Ubuntu-dialogue-corpus/*
!mkdir ubuntu-data ubuntu-npz

951M	Ubuntu-dialogue-corpus/dialogueText_196.csv
1.7G	Ubuntu-dialogue-corpus/dialogueText_301.csv
111M	Ubuntu-dialogue-corpus/dialogueText.csv


In [0]:
DF = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText.csv', parse_dates=['date'], chunksize=1200000)
DF_196 = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText_196.csv', parse_dates=['date'], chunksize=1200000)
DF_301 = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText_301.csv', parse_dates=['date'], chunksize=1200000)

Creating corpus from files.

In [0]:
def clean(dataset, file_name):
  i = 1
  for data in dataset:    
    text_corpus = ''
    current = None
    for msg in data.itertuples():
      if msg.dialogueID != current:
        current = msg.dialogueID
        text_corpus += '\n\n'
      try: 
        text_corpus += f"({msg.date}) {msg._4}: {msg.text}\n"
      except KeyError:
        pass
    
    #save data to file 
    with open(f'ubuntu-data/{file_name}_{i}.txt', 'w') as f:
      f.write(text_corpus)  
    del(text_corpus)
    i += 1

In [0]:
#clean(DF, 'cleaned_DF')
#clean(DF_196, 'cleaned_DF_196')
clean(DF_301, 'cleaned_301_DF')

Encoding

In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_1.txt ubuntu-npz/cleaned_301_DF_1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_2.txt ubuntu-npz/cleaned_301_DF_2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_3.txt ubuntu-npz/cleaned_301_DF_3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_4.txt ubuntu-npz/cleaned_301_DF_4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_5.txt ubuntu-npz/cleaned_301_DF_5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_6.txt ubuntu-npz/cleaned_301_DF_6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_7.txt ubuntu-npz/cleaned_301_DF_7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_8.txt ubuntu-npz/cleaned_301_DF_8.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_9.txt ubuntu-npz/cleaned_301_DF_9.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/cleaned_301_DF_10.txt ubuntu-npz/cleaned_301_DF_10.txt.npz --model_name 345M

Reading files
100% 1/1 [03:46<00:00, 226.69s/it]
Writing ubuntu-npz/cleaned_301_DF_1.txt.npz
Reading files
100% 1/1 [03:57<00:00, 237.85s/it]
Writing ubuntu-npz/cleaned_301_DF_2.txt.npz
Reading files
100% 1/1 [03:50<00:00, 230.73s/it]
Writing ubuntu-npz/cleaned_301_DF_3.txt.npz
Reading files
100% 1/1 [03:43<00:00, 223.00s/it]
Writing ubuntu-npz/cleaned_301_DF_4.txt.npz
Reading files
100% 1/1 [03:51<00:00, 231.07s/it]
Writing ubuntu-npz/cleaned_301_DF_5.txt.npz
Reading files
100% 1/1 [04:16<00:00, 256.84s/it]
Writing ubuntu-npz/cleaned_301_DF_6.txt.npz
Reading files
100% 1/1 [04:10<00:00, 250.63s/it]
Writing ubuntu-npz/cleaned_301_DF_7.txt.npz
Reading files
100% 1/1 [03:38<00:00, 218.86s/it]
Writing ubuntu-npz/cleaned_301_DF_8.txt.npz
Reading files
100% 1/1 [03:42<00:00, 222.68s/it]
Writing ubuntu-npz/cleaned_301_DF_9.txt.npz
Reading files
100% 1/1 [04:57<00:00, 297.85s/it]
Writing ubuntu-npz/cleaned_301_DF_10.txt.npz


Training

In [0]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every 250 --stop_after 1501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-12 06:32:43.447987: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 06:32:43.448491: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x27ef640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 06:32:43.448536: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categoric

In [0]:
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.95 --model_name 345M



2019-10-12 06:53:42.376726: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-12 06:53:42.377008: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x26af640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-12 06:53:42.377063: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

